# Open-Source Formal Verification for Chisel

In this notebook, we illustrate some of the new formal verification features that are the topic of our [WOSET 2021 paper](https://woset-workshop.github.io/WOSET2021.html#article-3).

First we download the latest [Chisel](https://github.com/chipsalliance/chisel3) and [chiseltest](https://github.com/ucb-bar/chisel-testers2) snapshot release from the Sonatypes package repository:

In [ ]:
interp.repositories() ++= Seq(
  coursierapi.MavenRepository.of("https://oss.sonatype.org/content/repositories/snapshots")
)

In [ ]:
import $ivy.`edu.berkeley.cs::chisel3:3.5-SNAPSHOT`
import $ivy.`edu.berkeley.cs::chiseltest:0.5-SNAPSHOT`

In [ ]:
import chisel3._
import chisel3.stage._


Now we can declare a simple Chisel module:

In [ ]:
class KeepMax(width: Int) extends Module {
  val in = IO(Input(UInt(width.W)))
  val out = IO(Output(UInt(width.W)))

  val max = RegInit(0.U(width.W))
  when (in > max) {
    max := in
  }
  out := max
    
  // get the value of io.out from 1 cycle in the past
  val firstCycle = RegInit(true.B)
  firstCycle := false.B
  when(!firstCycle) {
    assert(out >= RegNext(out))
  }
}

## Compilation Flow
Chisel is a Scala library for RTL circuits. These circuits can be exported into HiFirrtl which represents a static RTL circuit with high level Chisel features like memories, bundles and vectors. The firrtl compiler lowers these features to a more simple LoFirrtl representation which can then be exported to Verilog.

In [ ]:
val compiler = new ChiselStage
val genKeepMax = ChiselGeneratorAnnotation(() => new KeepMax(8))

In [ ]:
// compile to optimized low firrtl
compiler.execute(Array("-E", "low-opt"), Seq(genKeepMax))

Go ahead and inspect the new `KeepMax.opt.lo.fir` file that was generated in the root directory. We can see how the `assert` is represented as a native firrtl statement with two input: enable and predicate.

```
assert(clock, geq(out, REG), and(not(firstCycle), not(reset)), "") : assert
```

The `assert` statement is triggered on every rising edge of the `clock` signal. It is only enabled when it is not the `firstCycle` and when `reset` is not active. If that is the case, then it will check that `out` is greater or equal to the value of the delay register (`REG`) that was inserted by `RegNext(out)`.

There is also a `printf` statement which gets added by the `chisel3.assert` function. This statement will be ignored by our formal tools. It is useful in simulation and since we are going to use a simulator to replay the counter example that a formal tool might provide to us, it will also be useful to debug the failure of a formal check.

## Compilation to Verilog
LoFirrtl is normally compiled down to a simple subset of the (System)Verilog standard which works with a wide range of synthesis and verification tools. Thus we could compile our fomral property down to a concurrent assertion which works with both [Verilator](https://github.com/verilator/verilator/) as well as [SymbiYosys](https://github.com/YosysHQ/SymbiYosys).

In [ ]:
// compile to system verilog
compiler.execute(Array("-E", "sverilog"), Seq(genKeepMax))

In [19]:
// generate SMTLib using yosys from SystemVerilog
os.proc("yosys", "-p", "read_verilog -sv KeepMax.sv ; proc ; write_smt2").call()